In [95]:
# importing neccessary libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Read the dataset
path = "tested.csv"
df = pd.read_csv(path)

# Preprocessing
df["Sex"] = df["Sex"].map({'male':0,'female':1})

print(df.shape)
df.head()

(418, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,S


In [96]:
# Defining features and target
feature = ["Pclass","Sex","Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]
X = df[feature].copy()
y = df["Survived"]

# Select categorical columns
categorical_cols = [cname for cname in X.columns if X[cname].nunique() < 10 and 
                        X[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

print(numerical_cols)
print(categorical_cols)

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
['Embarked']


In [97]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

encoded = preprocessor.fit_transform(X)
encoded = pd.DataFrame(encoded)
encoded.head()

,0,1,2,3,4,5,6,7,8
0,3.0,0.0,34.5,0.0,0.0,7.8292,0.0,1.0,0.0
1,3.0,1.0,47.0,1.0,0.0,7.0000,0.0,0.0,1.0
2,2.0,0.0,62.0,0.0,0.0,9.6875,0.0,1.0,0.0
3,3.0,0.0,27.0,0.0,0.0,8.6625,0.0,0.0,1.0
4,3.0,1.0,22.0,1.0,1.0,12.2875,0.0,0.0,1.0


In [98]:
# Splitting datset into training and testing
X_train,X_test ,y_train , y_test = train_test_split(encoded, y, test_size=0.2)

In [99]:
# Defining out ML model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [100]:
# Evaluation of the model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_pred, y_test)
print("y_pred",y_pred[:5])
print("y_test",y_test[:5])

y_pred [0. 1. 0. 0. 0.]
y_test 194    0
225    1
56     0
130    0
243    0
Name: Survived, dtype: int64
